# RNN by Tensorflow

### 데이터 준비

In [1]:
VOCAB_SIZE = 300
SEQ_LEN = 100

In [ ]:
from tensorflow.keras.datasets import imdb

(train_input, train_target), (test_input, test_target) =  imdb.load_data(num_words=VOCAB_SIZE)
train_input.shape, train_target.shape, test_input.shape, test_target.shape

In [ ]:
train_input, test_input = train_input[:10000], test_input[:5000]
train_target, test_target = train_target[:10000], test_target[:5000]
train_input.shape, train_target.shape, test_input.shape, test_target.shape

In [ ]:
# padding 처리 (동일한 길이)
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input, maxlen=SEQ_LEN)
test_seq = pad_sequences(test_input, maxlen=SEQ_LEN)

train_seq.shape, test_seq.shape

In [ ]:
# one-hot encoding
from tensorflow.keras.utils import to_categorical

train_onehot = to_categorical(train_seq)
test_onehot = to_categorical(test_seq)

train_onehot.shape, test_onehot.shape

### 모델 준비 및 학습

In [ ]:
from tensorflow.keras import models, layers, callbacks, optimizers

input = layers.Input(shape=(SEQ_LEN, VOCAB_SIZE)) # 입력층
x = layers.SimpleRNN(units=8)(input) # 은닉층
output = layers.Dense(1, activation='sigmoid')(x) # 출력층

model = models.Model(input, output)

model.summary()

In [ ]:
# 컴파일 및 학습
import pandas as pd

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping_cb = callbacks.EarlyStopping(patience=7, restore_best_weights=True, verbose=1)
reduce_lr_on_plateau_cb = callbacks.ReduceLROnPlateau(patience=3, factor=0.7, verbose=1)

history = model.fit(
    train_onehot,
    train_target,
    epochs=100,
    batch_size=65,
    validation_split=0.2,
    callbacks=[early_stopping_cb, reduce_lr_on_plateau_cb])

history_df = pd.DataFrame(history.history)
history_df.plot()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_onehot, test_target)
loss, accuracy